In [1]:
import numpy as np
import Triangulation as tr
import HierarchicalSmooth as hs
import Base as base
from scipy.sparse import csc_matrix
import time

In [9]:
import plotly as py
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import graph_objs

py.offline.init_notebook_mode()

from scipy.spatial import Delaunay

import Base as base

AttributeError: 'module' object has no attribute 'Cryptography_HAS_SSL_ST'

In [3]:
filePrefix = '/home/siddharth/CMU/intro_to_research/intro/DREAM3D/DREAM3D-6.1.3-Linux-x86_64/Data/Mesh'

P = np.loadtxt( filePrefix + '/SharedVertexList.txt' ).T
tri = np.loadtxt( filePrefix + '/SharedTriList.txt', dtype=int )
nFaceLabels = np.loadtxt( filePrefix + '/FaceLabels.txt', dtype=int )
nType = np.loadtxt( filePrefix + '/NodeType.txt', dtype=int )

In [4]:
BoundaryDict = {}  # hashing surface elements takes a few minutes; better now than later!
for i in range( len ( nFaceLabels ) ):
    if nFaceLabels[i,0] > nFaceLabels[i,1]:
        nFaceLabels[i,0], nFaceLabels[i,1] = nFaceLabels[i,1], nFaceLabels[i,0]
        tri[i,0], tri[i,1] = tri[i,1], tri[i,0]
    try:
        BoundaryDict[ str( nFaceLabels[i] ) ].append( i )
    except KeyError:
        BoundaryDict[ str( nFaceLabels[i] ) ] = [ i ] # start a new list

nUniqFaces = np.vstack( { tuple( row ) for row in nFaceLabels} )

In [5]:
print 'Number of grain boundaries = %d' % len( BoundaryDict )
print 'Number of unique grains = %d' % np.unique( nUniqFaces ).size

Number of grain boundaries = 4728
Number of unique grains = 795


In [ ]:
nGB = 1

# triGB = tri[ np.where( base.ismember( nFaceLabels, nUniqFaces[nGB].reshape( 1, -1 ), 'rows' )[0] )[0], : ]
triGB = tri[ BoundaryDict[ str( nUniqFaces[nGB] ) ], : ]
pointGB = np.unique( triGB )
triGB = np.array( base.ismember( triGB, pointGB )[1] ).reshape( -1, 3 )
xGB = P[ :, pointGB ]
typeGB = nType[ pointGB ]

t0 = time.time()
FB, FBsec = hs.DifferentiateFaces( tr.Triangulation( triGB, xGB ) )
print 'Differentiating faces %s seconds' % str( time.time() - t0 )

In [14]:
%autoreload
nGrain = np.array( [ 182, 146, 23 ] )
nGrainPatches = np.any( base.ismember( nFaceLabels, nGrain )[0], axis=1 )
nGrainFaces = nFaceLabels[ nGrainPatches ]
triGrain = tri[ nGrainPatches, : ]
pointGrain = np.unique( triGrain )
triGrain = np.array( base.ismember( triGrain, pointGrain )[1] ).reshape( -1, 3 )
PGrain = P[ :, pointGrain ]
nGrainType = nType[ pointGrain ]

print 'Patches = %d' % len( triGrain )
print 'nodes = %d' % PGrain.shape[1]
# nFaceLabels[ np.any( nFaceLabels==nGrain, axis=1 ) ]

Patches = 3760
nodes = 1884


In [ ]:
fig1 = FF.create_trisurf(x=PGrain[0], y=PGrain[1], z=PGrain[2],
                         simplices=triGrain,
                         title="System of %d grains" % nGrain.size, 
                         aspectratio=dict( 
                            x = np.ptp( PGrain[0] ), 
                            y = np.ptp( PGrain[1] ), 
                            z = np.ptp( PGrain[2] )
                        )
                        )
py.offline.iplot(fig1 )

In [12]:
%autoreload
xS, bPS = hs.HierarchicalSmooth( PGrain, triGrain, nGrainFaces, nGrainType )

Hashing interface elements (this may take some time)... 1.3663380146 seconds. 
Interface [146 257]: 1 of 39...

HierarchicalSmooth.Smooth warning: Max. number of iterations reached.
HierarchicalSmooth.Smooth warning: Max. number of iterations reached.


 44.6793279648 seconds. 
Interface [ 23 748]: 2 of 39... 1.12443709373 seconds. 
Interface [122 182]: 3 of 39... 0.377675056458 seconds. 
Interface [135 182]: 4 of 39... 1.14991497993 seconds. 
Interface [182 422]: 5 of 39... 0.940422058105 seconds. 
Interface [169 182]: 6 of 39...

HierarchicalSmooth.Smooth warning: Max. number of iterations reached.


 22.5697309971 seconds. 
Interface [146 398]: 7 of 39... 0.725904941559 seconds. 
Interface [146 317]: 8 of 39... 0.959426879883 seconds. 
Interface [146 657]: 9 of 39... 1.22398996353 seconds. 
Interface [ 23 301]: 10 of 39... 0.966820001602 seconds. 
Interface [ 23 231]: 11 of 39... 0.458081960678 seconds. 
Interface [182 639]: 12 of 39... 1.22573685646 seconds. 
Interface [ -1 146]: 13 of 39...

HierarchicalSmooth.Smooth warning: Max. number of iterations reached.


 24.0119428635 seconds. 
Interface [146 579]: 14 of 39... 0.814415931702 seconds. 
Interface [ 11 182]: 15 of 39... 0.527235031128 seconds. 
Interface [146 259]: 16 of 39... 0.237420797348 seconds. 
Interface [146 715]: 17 of 39... 1.14065003395 seconds. 
Interface [146 697]: 18 of 39... 1.12144899368 seconds. 
Interface [182 311]: 19 of 39... 0.917966127396 seconds. 
Interface [182 368]: 20 of 39... 0.650207996368 seconds. 
Interface [146 352]: 21 of 39... 0.952342987061 seconds. 
Interface [182 541]: 22 of 39... 0.859292984009 seconds. 
Interface [  1 182]: 23 of 39... 0.357870101929 seconds. 
Interface [ 23 353]: 24 of 39... 0.898932933807 seconds. 
Interface [142 146]: 25 of 39... 0.116030931473 seconds. 
Interface [ 79 182]: 26 of 39... 1.28814411163 seconds. 
Interface [182 565]: 27 of 39... 1.10059690475 seconds. 
Interface [146 536]: 28 of 39... 0.689841985703 seconds. 
Interface [146 391]: 29 of 39... 0.537807941437 seconds. 
Interface [146 486]: 30 of 39... 0.801457881927 sec

In [13]:
np.savetxt( filePrefix + '/SmoothVertexList.txt', xS.T, fmt='%f' )
np.savetxt( filePrefix + '/SmoothTriList.txt', triGrain, fmt='%d')
np.savetxt( filePrefix + '/BoolSmoothed.txt', bPS, fmt='%d' )
np.savetxt( filePrefix + '/TypeSmoothed.txt', nGrainType, fmt='%d' )

In [10]:
%autoreload
triBad = tri[ BoundaryDict[ '[182 565]' ], : ]
pBad = np.unique( triBad )
triBad = np.array( base.ismember( triBad, pBad )[1] ).reshape( -1, 3 )
pointBad = P[ :, pBad ]
typeBad = nType[ pBad ]

T = tr.Triangulation( triBad, pointBad )
FB, FBsec = hs.DifferentiateFaces( T )
thisFB = FBsec[0]
thisRange = FB[ thisFB[0]:thisFB[1]+1, 0 ]

# pointPerimeter = pointBad[ :, thisRange ]
# fixPerimeter = np.where( ( typeBad[ thisRange ] )%10==4 )[0]

# xS = hs.Smooth( pointPerimeter.T, 1.e-4, 2000, 
#                nFixed=fixPerimeter
#               )[0]
# np.savetxt( filePrefix + '/Perimeter.txt', pointPerimeter, fmt='%f' )
# np.savetxt( filePrefix + '/Fixed.txt', fixPerimeter, fmt='%d' )
# np.savetxt( filePrefix + '/PSmooth.txt', xS.T, fmt='%f' )

xS, PS = hs.HierarchicalSmooth( 
    xPoints=pointBad, 
    tri=triBad, 
    nFaceLabels=np.array( [ 182, 667 ] ).reshape( 1, -1 ).repeat( len( triBad ), axis=0 ),
    nNodeType=typeBad, 
    fThreshold=1.e-5  
)

np.savetxt( filePrefix + '/BadVertex.txt', pointBad.T, fmt='%f' )
np.savetxt( filePrefix + '/BadTri.txt', triBad, fmt='%d' )
np.savetxt( filePrefix + '/BadType.txt', typeBad, fmt='%d' )
np.savetxt( filePrefix + '/BadSmooth.txt', xS.T, fmt='%f' )
np.savetxt( filePrefix + '/BadBool.txt', PS, fmt='%d' )

# print triBad.shape
# print pBad.shape
# # print nType[ pBad ]

# T = tr.Triangulation( triBad, pointBad )
# print hs.DifferentiateFaces( T )

Hashing interface elements (this may take some time)... 0.0732848644257 seconds. 
Interface [182 667]: 1 of 1...

/usr/local/lib/python2.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:247: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/usr/local/lib/python2.7/dist-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


 2.45848798752 seconds. 
